In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision
import tarfile
import torchvision.utils
from torchvision.datasets.utils import download_url
from torch.utils.data import random_split

from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor
from models import *
import torchvision.transforms as transforms
from torch.utils.data.dataloader import DataLoader
import gc
from utils import * 
from torchvision.utils import save_image
import time

In [2]:
# the path of the originalimages and the saved models
data_dir = './input/gender-classification-dataset'
folder_path1 = 'D:/gende recog adv ex/final networks'

In [3]:
transform = transforms.Compose([transforms.Resize((100, 100)), transforms.ToTensor()])
batch_size = 128

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() 
                                  else "cpu")
device

In [4]:
# the data loader of the test data set.
dataset_val = ImageFolder(data_dir+"/Validation", transform = transform)
test_dl = DataLoader(dataset_val, batch_size*2, num_workers=4, pin_memory=True)
test_loader = DeviceDataLoader(test_dl, device)


In [10]:
# read the provider model
original_model = GenCnnModel()
original_model = to_device(GenCnnModel(), device)
original_model.load_state_dict(torch.load(folder_path1+'model_main_new.pth'))
to_device(original_model, device);

<All keys matched successfully>

In [12]:
# the test data loader to check if the image was classified correctly or miscalssified

new_batch_size = 1
our_test_loader = DataLoader(dataset_val, new_batch_size, num_workers=0, pin_memory=True)

In [13]:
# get the list of wrongly classified images
final_acc, wrong_examples, labels, logits = test(original_model, device, our_test_loader)

C:\Users\012709558\AppData\Local\Continuum\anaconda3\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [14]:
wrong_labels = [logits[i].max(1, keepdim=True)[1].item() for i in range(len(labels))]

In [17]:
# read the surrogate mode, here you can change the name of the surrogate model to test the different surrogates.
explainable_model1 = exp_GenCnnModel()
explainable_model1 = to_device(exp_GenCnnModel(), device)
explainable_model1.load_state_dict(torch.load(folder_path+'/networks_final/Surrogate_1.pth'))

<All keys matched successfully>

In [18]:
# add the perturbation using the gradients of the surrogate model to the image and create the counterfactual example.
corrected_examples = np.zeros([len(wrong_examples)])
start = int(round(time.time()*1000))
perturbed_examples = []
new_labels = []
eps_all = []
for i in range(len(wrong_examples)):
    x, y, prediction = wrong_examples[i], labels[i], logits[i]
    eps = 0.0
#     print(i)
    while True:
        perturbed_image = x.clone()
        perturbed_image.requires_grad = True
        output = explainable_model1(perturbed_image)
        loss = F.nll_loss(output, y)
        explainable_model1.zero_grad()
            # Calculate gradients of model in backward pass
        loss.backward()
        img_grad = perturbed_image.grad.data
        perturbed_image = perturbed_image - eps*img_grad
        output = original_model(perturbed_image)
        new_label = output.max(1, keepdim=True)[1]
        if(new_label.item() == y.item()):
            perturbed_examples.append(perturbed_image.squeeze().data.cpu().numpy())
            new_labels.append(new_label)
            eps_all.append(eps)
            corrected_examples[i] = 1
            print("Image {} has been modified with epsilon {}".format(i, eps))
            break
        eps += 0.05
        if eps > 0.99:
            corrected_examples[i] = 0
#             perturbed_examples.append(x.squeeze().data.cpu().numpy())
#             new_labels.append(y)
#             eps_all.append(eps)
#             corrected_examples[i] = 1
            break
time_elapsed = int(round(time.time()*1000)) -start
print ('test time elapsed {}ms'.format(time_elapsed))
print ('test time elapsed {}s'.format(time_elapsed/1000))

Image 0 has been modified with epsilon 0.15000000000000002
Image 1 has been modified with epsilon 0.05
Image 2 has been modified with epsilon 0.25
Image 4 has been modified with epsilon 0.6
Image 5 has been modified with epsilon 0.25
Image 6 has been modified with epsilon 0.1
Image 7 has been modified with epsilon 0.05
Image 8 has been modified with epsilon 0.2
Image 9 has been modified with epsilon 0.1
Image 10 has been modified with epsilon 0.5499999999999999
Image 11 has been modified with epsilon 0.39999999999999997
Image 12 has been modified with epsilon 0.05
Image 13 has been modified with epsilon 0.1
Image 15 has been modified with epsilon 0.25
Image 17 has been modified with epsilon 0.1
Image 18 has been modified with epsilon 0.1
Image 19 has been modified with epsilon 0.1
Image 20 has been modified with epsilon 0.15000000000000002
Image 21 has been modified with epsilon 0.1
Image 22 has been modified with epsilon 0.3
Image 23 has been modified with epsilon 0.15000000000000002


Image 208 has been modified with epsilon 0.65
Image 209 has been modified with epsilon 0.25
Image 210 has been modified with epsilon 0.3
Image 211 has been modified with epsilon 0.8000000000000002
Image 212 has been modified with epsilon 0.3
Image 213 has been modified with epsilon 0.44999999999999996
Image 214 has been modified with epsilon 0.25
Image 215 has been modified with epsilon 0.35
Image 217 has been modified with epsilon 0.25
Image 218 has been modified with epsilon 0.25
Image 219 has been modified with epsilon 0.05
Image 220 has been modified with epsilon 0.05
Image 221 has been modified with epsilon 0.65
Image 222 has been modified with epsilon 0.39999999999999997
Image 223 has been modified with epsilon 0.1
Image 225 has been modified with epsilon 0.05
Image 226 has been modified with epsilon 0.49999999999999994
Image 227 has been modified with epsilon 0.05
Image 228 has been modified with epsilon 0.25
Image 229 has been modified with epsilon 0.44999999999999996
Image 230

In [19]:
# declaring the lists with the original image, the corresponding label, and the perturbations.
real_examples = []
real_labels = []
wrong_predictions = []
corrected_idx = np.where(corrected_examples == 1)
for idx in corrected_idx[0]:
    real_examples.append(wrong_examples[idx].squeeze().data.cpu().numpy())
    real_labels.append(labels[idx].item())
    wrong_predictions.append(wrong_labels[idx])

In [20]:
def get_quartlies(samples):
    q1, med, q3 = np.percentile(samples, [25, 50, 75])
    return q3, q3-q1

In [21]:
diff = []
tau = 15
for i in range(len(eps_all)):
    diff.append((real_examples[i] - perturbed_examples[i])**2)
    q3, iqr = get_quartlies(diff[i])
    idx = np.where(diff[i] < q3+iqr*tau)
    diff[i][idx]*=0

In [22]:
import numpy as np
import matplotlib.pyplot as plt

In [23]:
# boosting the important pixels and neglecting the prxels with low importance.
diff2 = diff
for im in range(len(diff)):
    for layer in range(3):
        for color in range(100):
            for pixel in range(100):
                if diff[im][layer][color][pixel] > 0.0:
                    diff2[im][layer][color][pixel] = 1
                                

                

In [24]:
len(corrected_examples)

301

In [25]:
# the path to save the explanations of the wrongly classified images.
path_wrong = 'D:/gende recog adv ex/output images/Gender first surrogate/wrong_pri'

In [26]:
# normalizing the perturbations
for i in range(len(perturbed_examples)):
    for j in range(len(perturbed_examples[i])):
        for k in range(len(perturbed_examples[i][j])):
            for d in range(len(perturbed_examples[i][j][k])):
                perturbed_examples[i][j][k][d] = (perturbed_examples[i][j][k][d]-perturbed_examples[i].min())/(perturbed_examples[i].max()-perturbed_examples[i].min())


In [27]:
# save the original image and the perturbed image 
for i in range(len(wrong_predictions)):
    z = torch.from_numpy(real_examples[i])
    image1 = z.permute(1,2,0)
    image = np.float32(image1)
    z = torch.from_numpy(diff[i])
    image3 = z.permute(1,2,0)
    image3 = np.float32(image3+image1)
    for j in range(len(image3)):
        for k in range(len(image3[j])):
            for d in range(len(image3[j][k])):
                if image3[j][k][d] > 1:
                    image3[j][k][d] = 1

    plt.imsave(path_wrong+'/image_with_exp/'+str(i)+"label is"+str(state[real_labels[i]])+".jpg",image3)

In [29]:
# get the list of correctly classified images
final_acc, wrong_examples, labels, logits = test_images_true_classified(original_model, device, our_test_loader)

In [30]:
wrong_labels = [logits[i].max(1, keepdim=True)[1].item() for i in range(len(labels))]

In [31]:
len(wrong_labels)

11256

In [34]:
# add the perturbation using the gredients of the surrogate model to the image and create the counterfactual example.
corrected_examples = np.zeros([len(wrong_examples)])
start = int(round(time.time()*1000)) 
perturbed_examples = []
new_labels = []
eps_all = []
for i in range(len(wrong_examples)):
    x, y, prediction = wrong_examples[i], labels[i], logits[i]
#     print(prediction.argmax())
    if prediction.argmax() == 0:
        y = 1
        y =torch.tensor([y])
    else:
        y = 0
        y =torch.tensor([y])
    y = y.to(device)
    eps = 0.0
#     print(i)
    while True:
        perturbed_image = x.clone()
        perturbed_image.requires_grad = True
        output = explainable_model1(perturbed_image)
        loss = F.nll_loss(output, y)
        explainable_model1.zero_grad()
            # Calculate gradients of model in backward pass
        loss.backward()
        img_grad = perturbed_image.grad.data
        perturbed_image = perturbed_image - eps*img_grad
        output = original_model(perturbed_image)
        new_label = output.max(1, keepdim=True)[1]
        if(new_label.item() == y.item()):
            perturbed_examples.append(perturbed_image.squeeze().data.cpu().numpy())
            new_labels.append(new_label)
            eps_all.append(eps)
            corrected_examples[i] = 1
            print("Image {} has been modified with epsilon {}".format(i, eps))
            break
        eps += 0.05
        if eps > 0.99:
            corrected_examples[i] = 0
#             perturbed_examples.append(x.squeeze().data.cpu().numpy())
#             new_labels.append(y)
#             eps_all.append(eps)
#             corrected_examples[i] = 1
            break
    
time_elapsed = int(round(time.time()*1000)) -start
print ('test time elapsed {}ms'.format(time_elapsed))
print ('test time elapsed {}s'.format(time_elapsed/1000))

Image 0 has been modified with epsilon 0.5499999999999999
Image 6 has been modified with epsilon 0.35
Image 12 has been modified with epsilon 0.39999999999999997
Image 15 has been modified with epsilon 0.7000000000000001
Image 21 has been modified with epsilon 0.44999999999999996
Image 24 has been modified with epsilon 0.9000000000000002
Image 34 has been modified with epsilon 0.35
Image 35 has been modified with epsilon 0.2
Image 47 has been modified with epsilon 0.39999999999999997
Image 51 has been modified with epsilon 0.9000000000000002
Image 56 has been modified with epsilon 0.9000000000000002
Image 58 has been modified with epsilon 0.39999999999999997
Image 61 has been modified with epsilon 0.5499999999999999
Image 66 has been modified with epsilon 0.8500000000000002
Image 67 has been modified with epsilon 0.1
Image 71 has been modified with epsilon 0.44999999999999996
Image 73 has been modified with epsilon 0.8000000000000002
Image 74 has been modified with epsilon 0.4999999999

Image 504 has been modified with epsilon 0.49999999999999994
Image 505 has been modified with epsilon 0.8500000000000002
Image 506 has been modified with epsilon 0.7000000000000001
Image 508 has been modified with epsilon 0.49999999999999994
Image 509 has been modified with epsilon 0.44999999999999996
Image 511 has been modified with epsilon 0.65
Image 516 has been modified with epsilon 0.7000000000000001
Image 520 has been modified with epsilon 0.49999999999999994
Image 522 has been modified with epsilon 0.5499999999999999
Image 524 has been modified with epsilon 0.3
Image 531 has been modified with epsilon 0.39999999999999997
Image 535 has been modified with epsilon 0.9000000000000002
Image 547 has been modified with epsilon 0.6
Image 553 has been modified with epsilon 0.8000000000000002
Image 555 has been modified with epsilon 0.49999999999999994
Image 556 has been modified with epsilon 0.44999999999999996
Image 557 has been modified with epsilon 0.39999999999999997
Image 561 has be

Image 948 has been modified with epsilon 0.49999999999999994
Image 952 has been modified with epsilon 0.15000000000000002
Image 953 has been modified with epsilon 0.44999999999999996
Image 954 has been modified with epsilon 0.9500000000000003
Image 956 has been modified with epsilon 0.5499999999999999
Image 960 has been modified with epsilon 0.9000000000000002
Image 961 has been modified with epsilon 0.9000000000000002
Image 966 has been modified with epsilon 0.9500000000000003
Image 968 has been modified with epsilon 0.7500000000000001
Image 970 has been modified with epsilon 0.35
Image 974 has been modified with epsilon 0.39999999999999997
Image 976 has been modified with epsilon 0.9000000000000002
Image 979 has been modified with epsilon 0.6
Image 981 has been modified with epsilon 0.9000000000000002
Image 983 has been modified with epsilon 0.9000000000000002
Image 984 has been modified with epsilon 0.1
Image 987 has been modified with epsilon 0.49999999999999994
Image 989 has been 

Image 1390 has been modified with epsilon 0.7000000000000001
Image 1401 has been modified with epsilon 0.25
Image 1405 has been modified with epsilon 0.7500000000000001
Image 1414 has been modified with epsilon 0.9000000000000002
Image 1415 has been modified with epsilon 0.5499999999999999
Image 1417 has been modified with epsilon 0.15000000000000002
Image 1435 has been modified with epsilon 0.8000000000000002
Image 1437 has been modified with epsilon 0.9500000000000003
Image 1438 has been modified with epsilon 0.35
Image 1440 has been modified with epsilon 0.8500000000000002
Image 1442 has been modified with epsilon 0.8000000000000002
Image 1445 has been modified with epsilon 0.8000000000000002
Image 1447 has been modified with epsilon 0.9000000000000002
Image 1455 has been modified with epsilon 0.5499999999999999
Image 1456 has been modified with epsilon 0.6
Image 1459 has been modified with epsilon 0.44999999999999996
Image 1461 has been modified with epsilon 0.3
Image 1462 has been

Image 1877 has been modified with epsilon 0.7000000000000001
Image 1880 has been modified with epsilon 0.5499999999999999
Image 1881 has been modified with epsilon 0.9000000000000002
Image 1886 has been modified with epsilon 0.1
Image 1887 has been modified with epsilon 0.2
Image 1888 has been modified with epsilon 0.44999999999999996
Image 1889 has been modified with epsilon 0.25
Image 1890 has been modified with epsilon 0.15000000000000002
Image 1894 has been modified with epsilon 0.1
Image 1901 has been modified with epsilon 0.7000000000000001
Image 1902 has been modified with epsilon 0.6
Image 1903 has been modified with epsilon 0.8000000000000002
Image 1904 has been modified with epsilon 0.35
Image 1906 has been modified with epsilon 0.35
Image 1907 has been modified with epsilon 0.49999999999999994
Image 1910 has been modified with epsilon 0.44999999999999996
Image 1911 has been modified with epsilon 0.65
Image 1913 has been modified with epsilon 0.6
Image 1915 has been modified 

...

Image 9070 has been modified with epsilon 0.9500000000000003
Image 9074 has been modified with epsilon 0.5499999999999999
Image 9075 has been modified with epsilon 0.6
Image 9076 has been modified with epsilon 0.39999999999999997
Image 9078 has been modified with epsilon 0.35
Image 9079 has been modified with epsilon 0.3
Image 9090 has been modified with epsilon 0.6
Image 9093 has been modified with epsilon 0.8000000000000002
Image 9094 has been modified with epsilon 0.8000000000000002
Image 9096 has been modified with epsilon 0.7500000000000001
Image 9097 has been modified with epsilon 0.9000000000000002
Image 9100 has been modified with epsilon 0.44999999999999996
Image 9101 has been modified with epsilon 0.2
Image 9108 has been modified with epsilon 0.15000000000000002
Image 9110 has been modified with epsilon 0.65
Image 9113 has been modified with epsilon 0.44999999999999996
Image 9118 has been modified with epsilon 0.9000000000000002
Image 9120 has been modified with epsilon 0.549

Image 9486 has been modified with epsilon 0.39999999999999997
Image 9487 has been modified with epsilon 0.35
Image 9494 has been modified with epsilon 0.8500000000000002
Image 9499 has been modified with epsilon 0.44999999999999996
Image 9500 has been modified with epsilon 0.5499999999999999
Image 9501 has been modified with epsilon 0.5499999999999999
Image 9504 has been modified with epsilon 0.1
Image 9506 has been modified with epsilon 0.6
Image 9507 has been modified with epsilon 0.35
Image 9510 has been modified with epsilon 0.3
Image 9513 has been modified with epsilon 0.7500000000000001
Image 9515 has been modified with epsilon 0.7000000000000001
Image 9525 has been modified with epsilon 0.6
Image 9527 has been modified with epsilon 0.6
Image 9530 has been modified with epsilon 0.2
Image 9531 has been modified with epsilon 0.8000000000000002
Image 9533 has been modified with epsilon 0.49999999999999994
Image 9534 has been modified with epsilon 0.44999999999999996
Image 9539 has b

Image 10019 has been modified with epsilon 0.44999999999999996
Image 10020 has been modified with epsilon 0.65
Image 10023 has been modified with epsilon 0.9000000000000002
Image 10026 has been modified with epsilon 0.7500000000000001
Image 10027 has been modified with epsilon 0.39999999999999997
Image 10028 has been modified with epsilon 0.8500000000000002
Image 10029 has been modified with epsilon 0.7000000000000001
Image 10030 has been modified with epsilon 0.49999999999999994
Image 10037 has been modified with epsilon 0.25
Image 10041 has been modified with epsilon 0.8000000000000002
Image 10047 has been modified with epsilon 0.39999999999999997
Image 10049 has been modified with epsilon 0.39999999999999997
Image 10051 has been modified with epsilon 0.44999999999999996
Image 10053 has been modified with epsilon 0.8500000000000002
Image 10057 has been modified with epsilon 0.3
Image 10058 has been modified with epsilon 0.49999999999999994
Image 10063 has been modified with epsilon 0

Image 10513 has been modified with epsilon 0.65
Image 10519 has been modified with epsilon 0.8500000000000002
Image 10521 has been modified with epsilon 0.6
Image 10522 has been modified with epsilon 0.1
Image 10528 has been modified with epsilon 0.49999999999999994
Image 10530 has been modified with epsilon 0.7000000000000001
Image 10531 has been modified with epsilon 0.35
Image 10537 has been modified with epsilon 0.3
Image 10538 has been modified with epsilon 0.2
Image 10546 has been modified with epsilon 0.44999999999999996
Image 10547 has been modified with epsilon 0.15000000000000002
Image 10550 has been modified with epsilon 0.35
Image 10552 has been modified with epsilon 0.65
Image 10553 has been modified with epsilon 0.15000000000000002
Image 10557 has been modified with epsilon 0.49999999999999994
Image 10559 has been modified with epsilon 0.7500000000000001
Image 10562 has been modified with epsilon 0.39999999999999997
Image 10569 has been modified with epsilon 0.49999999999

Image 10978 has been modified with epsilon 0.9000000000000002
Image 10980 has been modified with epsilon 0.5499999999999999
Image 10983 has been modified with epsilon 0.35
Image 10984 has been modified with epsilon 0.49999999999999994
Image 10985 has been modified with epsilon 0.2
Image 10986 has been modified with epsilon 0.65
Image 10987 has been modified with epsilon 0.9000000000000002
Image 10993 has been modified with epsilon 0.49999999999999994
Image 10994 has been modified with epsilon 0.39999999999999997
Image 10997 has been modified with epsilon 0.2
Image 11003 has been modified with epsilon 0.7000000000000001
Image 11005 has been modified with epsilon 0.6
Image 11006 has been modified with epsilon 0.44999999999999996
Image 11007 has been modified with epsilon 0.7500000000000001
Image 11008 has been modified with epsilon 0.7000000000000001
Image 11011 has been modified with epsilon 0.7000000000000001
Image 11014 has been modified with epsilon 0.49999999999999994
Image 11015 ha

In [35]:
# declaring the lists with the original image, the corresponding label, and the perturbations.
real_examples = []
real_labels = []
wrong_predictions = []
corrected_idx = np.where(corrected_examples == 1)
for idx in corrected_idx[0]:
    real_examples.append(wrong_examples[idx].squeeze().data.cpu().numpy())
    real_labels.append(labels[idx].item())
    wrong_predictions.append(wrong_labels[idx])

In [36]:
diff = []
tau = 15
for i in range(len(eps_all)):
    diff.append((real_examples[i] - perturbed_examples[i])**2)
    q3, iqr = get_quartlies(diff[i])
    idx = np.where(diff[i] < q3+iqr*tau)
    diff[i][idx]*=0

In [37]:
# boosting the important pixels and neglecting the prxels with low importance.
diff2 = diff
for im in range(len(diff)):
    for layer in range(3):
        for color in range(100):
            for pixel in range(100):
                if diff[im][layer][color][pixel] > 0.0:
                    diff2[im][layer][color][pixel] = 1
                                

                

In [39]:
the path to save the explanation of the correctly classified
path_wrong = 'D:/gende recog adv ex/output images/Gender first surrogate/wright_pri'

In [40]:
for i in range(len(perturbed_examples)):
    for j in range(len(perturbed_examples[i])):
        for k in range(len(perturbed_examples[i][j])):
            for d in range(len(perturbed_examples[i][j][k])):
                perturbed_examples[i][j][k][d] = (perturbed_examples[i][j][k][d]-perturbed_examples[i].min())/(perturbed_examples[i].max()-perturbed_examples[i].min())


In [41]:
for i in range(len(wrong_predictions)):
    z = torch.from_numpy(real_examples[i])
    image1 = z.permute(1,2,0)
    image1 = np.float32(image1)
    z = torch.from_numpy(diff[i])
    image3 = z.permute(1,2,0)
    image3 = np.float32(image3+image1)
    for j in range(len(image3)):
        for k in range(len(image3[j])):
            for d in range(len(image3[j][k])):
                if image3[j][k][d] > 1:
                    image3[j][k][d] = 1

    plt.imsave(path_wrong+'/image_with_exp/'+str(i)+"label is"+str(state[real_labels[i]])+".jpg",image3)

In [42]:
# #real_examples, perturbed_examples, diff, real_labels, wrong_predictions
# fig=plt.figure(figsize=(12, 800))
# columns = 3
# rows = len(wrong_predictions)
# c = 1
# for i in range(len(wrong_predictions)):
#     ax = fig.add_subplot(rows, columns, c)
#     ax.set_title('Predicted as {}'.format(wrong_predictions[i]))
#     ax.set_xticklabels([])
#     ax.set_yticklabels([])
#     ax.set_aspect('equal')
#     z =torch.from_numpy(real_examples[i])
#     image = z.permute(1,2,0)
#     plt.imshow(image)
#     ax = fig.add_subplot(rows, columns, c+1)
#     ax.set_title('Corrected image as {}'.format(real_labels[i]))
#     ax.set_xticklabels([])
#     ax.set_yticklabels([])
#     ax.set_aspect('equal')
#     z =torch.from_numpy(perturbed_examples[i])
#     image = z.permute(1,2,0)
#     plt.imshow(image)
#     ax = fig.add_subplot(rows, columns, c+2)
#     ax.set_title('Pixels cause')
#     ax.set_xticklabels([])
#     ax.set_yticklabels([])
#     ax.set_aspect('equal')
#     z =torch.from_numpy(diff[i])
#     image = z.permute(1,2,0)
#     plt.imshow(image, cmap='hot')
#     c = c + 3
# plt.subplots_adjust(wspace=0, hspace=0)
# plt.tight_layout(True)
# plt.show(block=True)